# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

burdan sonraki kısım ucak rezervasyonu için ekledim

In [12]:
# 1. Gerekli kütüphaneleri yükle
import os
import json
import random
import string
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [3]:
# 2. Ortam değişkenlerini yükle
load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-4o-mini"


In [4]:
# 3. Uçak bileti fiyatları sözlüğü (çift şehirli anahtarlarla)
ticket_prices = {
    ("istanbul", "berlin"): "$499",
    ("istanbul", "tokyo"): "$1400",
    ("ankara", "berlin"): "$550",
    ("izmir", "tokyo"): "$1350",
    ("antalya", "paris"): "$790",
    ("istanbul", "paris"): "$899"
}


In [5]:

# 4. Bilet fiyatı fonksiyonu (güvenli)
def get_ticket_price(departure_city, destination_city):
    if not departure_city or not destination_city:
        return "Error: both departure and destination cities are required."
    key = (departure_city.lower(), destination_city.lower())
    price = ticket_prices.get(key)
    return price if price else f"Estimated price from {departure_city.title()} to {destination_city.title()} is around $1000."

In [6]:

# 5. PNR üretici & rezervasyon fonksiyonu (tarih destekli)
def generate_pnr():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

def make_reservation(departure_city, destination_city, travel_date):
    pnr = generate_pnr()
    # travel_date format 'YYYY-MM-DD'
    try:
        fly_date = datetime.fromisoformat(travel_date)
        date_str = fly_date.strftime("%d %B %Y")
    except:
        date_str = travel_date
    return (
        f"Your reservation from {departure_city.title()} to {destination_city.title()} on {date_str} is confirmed.\n"
        f"Flight departs at 13:45 local time.\n"
        f"Reservation Code (PNR): {pnr}.\n"
        f"Seat selection will be available during check-in."
    )

In [7]:
# 6. Fonksiyon sarmalayıcıları
def get_ticket_price_fn(departure_city: str, destination_city: str) -> str:
    return get_ticket_price(departure_city, destination_city)

def make_reservation_fn(departure_city: str, destination_city: str, travel_date: str) -> str:
    return make_reservation(departure_city, destination_city, travel_date)


In [8]:

# 7. Fonksiyon tanımları
functions = [
    {
        "name": "get_ticket_price",
        "description": "Returns the ticket price between a departure and destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."}
            },
            "required": ["departure_city", "destination_city"]
        }
    },
    {
        "name": "make_reservation",
        "description": "Reserves a flight from departure to destination city on a given date and returns reservation details.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."},
                "travel_date": {"type": "string", "description": "Travel date in YYYY-MM-DD format."}
            },
            "required": ["departure_city", "destination_city", "travel_date"]
        }
    }
]

In [9]:
# 8. Sistem mesajı
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
    "Never call any functions unless both the departure city, destination city, and travel date are clearly provided. "
    "If the user mentions only destination or departure, ask for the missing details. "
    "Ask explicitly for travel date in YYYY-MM-DD format."
)


In [10]:

# 9. Ana sohbet fonksiyonu
def chat(message, history):
    msgs = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # Modeli çağır ve varsa fonksiyon isteği al
    response = openai.chat.completions.create(
        model=MODEL,
        messages=msgs,
        functions=functions
    )
    msg = response.choices[0].message
    # Eğer fonksiyon çağırıldıysa
    if msg.function_call:
        fname = msg.function_call.name
        args = json.loads(msg.function_call.arguments)
        # Parametre kontrolü
        if fname == "get_ticket_price":
            result = get_ticket_price_fn(**args)
        else:
            # Önce rezervasyon için tarih parametresini al
            result = make_reservation_fn(**args)
        # Fonksiyon çağrı ve yanıtını ekle
        msgs.append({"role": "assistant", "content": None, "function_call": msg.function_call.to_dict()})
        msgs.append({"role": "function", "name": fname, "content": json.dumps(result)})
        # Son cevabı al
        response = openai.chat.completions.create(model=MODEL, messages=msgs)
        return response.choices[0].message.content
    return msg.content


In [13]:
# 10. Gradio arayüzünü başlat
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


uçuş saati toolu

In [ ]:
# FlightAI Tool Calling Demo for Jupyter Notebook

# Cell 1: Imports and API setup
from dotenv import load_dotenv
import os
import json
import uuid
from openai import OpenAI
import gradio as gr


In [ ]:
# Load environment variables and initialize OpenAI client
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OpenAI API key not found in environment")
openai = OpenAI()
MODEL = "gpt-4o-mini"


In [ ]:
# Cell 2: Demo Data
# Ticket prices for demonstration purposes
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
# Flight schedule data without dates for simplicity
flight_schedule = [
    {"from": "istanbul", "to": "paris", "time": "08:30"},
    {"from": "istanbul", "to": "paris", "time": "15:45"},
    {"from": "istanbul", "to": "berlin", "time": "09:15"},
    {"from": "ankara", "to": "paris", "time": "11:00"}
]


In [ ]:
# Cell 3: Tool Functions
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    return ticket_prices.get(destination_city.lower(), "Unknown")

def get_flight_time(departure_city, destination_city):
    print(f"Tool get_flight_time called for {departure_city}->{destination_city}")
    dep = departure_city.lower()
    dest = destination_city.lower()
    for f in flight_schedule:
        if f["from"] == dep and f["to"] == dest:
            return f["time"]
    return "No flights found"

def book_flight(departure_city, destination_city, date, passenger_name):
    print(f"Tool book_flight called for {departure_city}->{destination_city} on {date} for {passenger_name}")
    time = get_flight_time(departure_city, destination_city)
    price = get_ticket_price(destination_city)
    if time.startswith("No"):
        return {"status": "error", "message": "No flights available"}
    pnr = str(uuid.uuid4())[:6].upper()
    return {
        "status": "ok",
        "departure_city": departure_city,
        "destination_city": destination_city,
        "date": date,
        "time": time,
        "price": price,
        "passenger_name": passenger_name,
        "pnr": pnr
    }

In [ ]:
# Cell 4: Tool Schemas
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {"type": "string", "description": "The city that the customer wants to travel to"}
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

flight_time_function = {
    "name": "get_flight_time",
    "description": "Get the departure time for a flight between two cities.",
    "parameters": {
        "type": "object",
        "properties": {
            "departure_city": {"type": "string", "description": "The city the passenger is flying from"},
            "destination_city": {"type": "string", "description": "The city the passenger wants to fly to"}
        },
        "required": ["departure_city", "destination_city"],
        "additionalProperties": False
    }
}

book_flight_function = {
    "name": "book_flight",
    "description": "Book a flight given departure, destination, date and passenger name; returns booking details.",
    "parameters": {
        "type": "object",
        "properties": {
            "departure_city": {"type": "string", "description": "Departure city"},
            "destination_city": {"type": "string", "description": "Destination city"},
            "date": {"type": "string", "description": "Travel date in YYYY-MM-DD"},
            "passenger_name": {"type": "string", "description": "Name of the passenger"}
        },
        "required": ["departure_city", "destination_city", "date", "passenger_name"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": flight_time_function},
    {"type": "function", "function": book_flight_function}
]

In [ ]:
# Cell 5: System Message
system_message = (
    "You are a FlightAI assistant. When the user provides departure, destination, and date, "
    "first call get_flight_time and get_ticket_price to fetch time and price, "
    "then ask for confirmation and passenger name. Only after confirmation, call book_flight."
)


In [ ]:
# Cell 6: Tool Call Handler
def handle_tool_call(tool_call):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    if name == "get_ticket_price":
        result = get_ticket_price(args["destination_city"])
    elif name == "get_flight_time":
        result = get_flight_time(args["departure_city"], args["destination_city"])
    elif name == "book_flight":
        result = book_flight(
            args["departure_city"],
            args["destination_city"],
            args["date"],
            args["passenger_name"]
        )
    else:
        result = {}
    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    }


In [ ]:
# Cell 7: Chat Function
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    # Handle all tool calls
    while response.choices[0].finish_reason == "tool_calls":
        tool_msg = response.choices[0].message
        messages.append(tool_msg)
        for tc in tool_msg.tool_calls:
            tool_response = handle_tool_call(tc)
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    return response.choices[0].message.content

In [23]:

# Cell 8: Launch Gradio Interface
gr.ChatInterface(fn=chat, type="messages", title="FlightAI Demo").launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Tool get_flight_time called for Istanbul->Paris
Tool get_ticket_price called for Paris
Tool book_flight called for Istanbul->Paris on 2025-08-12 for Ali Veli
Tool get_flight_time called for Istanbul->Paris
Tool get_ticket_price called for Paris
Tool book_flight called for Istanbul->Paris on 2025-08-12 for Ali Veli
Tool get_flight_time called for Istanbul->Paris
Tool get_ticket_price called for Paris
Tool book_flight called for Istanbul->Paris on 2025-08-12 for Maria Veli
Tool get_flight_time called for Istanbul->Paris
Tool get_ticket_price called for Paris
